<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.63
    Uninstalling yfinance-0.2.63:
      Successfully uninstalled yfinance-0.2.63
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-06-27 16:07:23
-------------------
qualified stocks: 93
with latest results: 29
still star stocks: 18
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  6.47 K
Current:  1.34 C
-------------------
Today PnL: 26.02 K (0.19%)
Current PnL: -15.26 L (-10.63%)
CY Booked + Current PnL: -8.52 L (-5.93%)
-------------------
Total profit:  3.60 L
Total loss:  -18.86 L
-------------------
Total Booked + Current PnL: 18.46 L (15.84%)
Total Booked PnL: 33.72 L (28.93%)
Curr Year Booked PnL: 6.74 L (5.02%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.07 C
Est FTT PnL: 72.57 L (54.04%)
Deployed:  1.17 C
Current:  1.34 C
CAGR/XIRR %: 9.72%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ADANIPORTS,1103.69,1583.00,1.67,M-LC,52.69,102254.0,23892.0,10144.0,0.76,30.49,9.92,43.43,77.0,2.36,0.76,37.43,XY24,NTT,MISC
36,HINDALCO,651.95,761.55,-19.08,H-LC,74.19,111576.0,7264.0,10276.0,0.98,6.96,9.21,16.81,27.0,0.71,0.83,24.13,X5K,ATH,METALS
84,UJJIVANSFB,52.77,53.00,51.69,M-SC,83.87,131733.0,-10746.0,11369.0,-0.63,-7.54,8.63,0.44,243.0,-0.95,0.98,55.48,OX40N,NTT,BANKS
78,TATAELXSI,7332.28,7332.00,-15.46,X-MC,38.71,88830.0,-13822.0,13822.0,1.12,-13.46,15.56,-0.00,58.0,-1.00,0.66,35.03,OX40N,NTT,IT
89,VIPIND,488.80,489.00,-212.78,H-SC,96.77,81354.0,-13962.0,14001.0,0.12,-14.65,17.21,0.04,154.0,-1.00,0.61,60.46,OX40N,NTT,MISC


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SAIL,130.64,228.00,88.63,M-MC,51.61,226787.0,1825.0,165827.0,-1.44,0.81,73.12,74.53,185.0,0.01,1.69,31.67,XY24,BTT,STEEL
12,BANKINDIA,116.91,190.00,-31.29,M-MC,55.91,179592.0,-216.0,112622.0,-0.15,-0.12,62.71,62.52,171.0,-0.00,1.34,33.93,XR,NTT,BANKS
51,KPIGREEN,497.21,732.16,21.90,H-SC,71.51,124765.0,-532.0,59737.0,-0.41,-0.42,47.88,47.25,139.0,-0.01,0.93,56.09,MH,ATH,POWER
35,HEROMOTOCO,4311.81,6039.03,-6.19,H-MC,33.33,151210.0,297.0,60151.0,0.99,0.20,39.78,40.06,102.0,0.00,1.13,23.68,AR,ATH,AUTO
27,DIXON,14367.00,18940.15,-37.09,H-MC,13.98,115784.0,848.0,35743.0,1.10,0.74,30.87,31.83,99.0,0.02,0.86,32.70,X40N,ATH,IT


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,JIOFIN,310.58,387.00,5.62,M-LC,89.25,226415.0,9009.0,44491.0,3.50,4.14,19.65,24.61,95.0,0.20,1.69,60.98,XY24,BTT,FINANCE
20,CERA,8421.60,8422.00,-14.08,X-SC,69.89,88608.0,-20873.0,20876.0,3.33,-19.07,23.56,0.00,68.0,-1.00,0.66,34.11,OX40N,NTT,CERAMICS
6,ASIANPAINT,2961.56,4250.00,-19.25,H-LC,48.39,160392.0,-40994.0,128602.0,3.01,-20.36,80.18,43.51,32.0,-0.32,1.19,11.65,X40,BTT,PAINTS
5,APOLLOHOSP,6904.43,8285.00,-19.51,H-LC,53.76,109628.0,6062.0,14646.0,2.92,5.85,13.36,20.00,30.0,0.41,0.82,20.75,X40N,BTT,HEALTHCARE
39,HONAUT,42646.15,58357.33,-17.46,X-MC,61.29,119040.0,-8898.0,56032.0,2.66,-6.96,47.07,36.84,56.0,-0.16,0.89,21.87,X40N,ATH,ELECTRICAL


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,RAJESHEXPO,517.65,518.00,1825.51,M-SC,37.63,52528.0,-84649.0,84743.0,-2.50,-61.71,161.33,0.07,231.0,-1.00,0.39,30.64,OX40N,NTT,JEWELLERY
23,COFFEEDAY,59.14,80.00,-51.71,L-SC,91.40,70618.0,-42931.0,82983.0,-2.36,-37.81,117.51,35.27,261.0,-0.52,0.53,72.84,XR,NTT,HOTELS
17,BSOFT,628.85,844.81,-10.11,M-SC,58.06,75635.0,-34414.0,72209.0,-2.26,-31.27,95.47,34.34,200.0,-0.48,0.56,24.52,XR,ATH,IT
15,BLUESTARCO,1646.70,2337.55,-13.40,H-SC,64.52,162400.0,-2270.0,71359.0,-2.22,-1.38,43.94,41.95,121.0,-0.03,1.21,5.98,X40N,ATH,AC
66,SAMMAANCAP,170.35,326.00,-172.73,M-SC,95.70,84414.0,-17796.0,111190.0,-1.87,-17.41,131.72,91.37,206.0,-0.16,0.63,37.03,XY25,NTT,FINANCE


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,UJJIVANSFB,52.77,53.0,51.69,M-SC,83.87,131733.0,-10746.0,11369.0,-0.63,-7.54,8.63,0.44,243.0,-0.95,0.98,55.48,OX40N,NTT,BANKS
78,TATAELXSI,7332.28,7332.0,-15.46,X-MC,38.71,88830.0,-13822.0,13822.0,1.12,-13.46,15.56,-0.00,58.0,-1.00,0.66,35.03,OX40N,NTT,IT
89,VIPIND,488.80,489.0,-212.78,H-SC,96.77,81354.0,-13962.0,14001.0,0.12,-14.65,17.21,0.04,154.0,-1.00,0.61,60.46,OX40N,NTT,MISC
76,SYMPHONY,1306.42,1306.0,-33.55,M-SC,1.08,141716.0,-29425.0,29364.0,0.13,-17.19,20.72,-0.03,189.0,-1.00,1.06,1.85,OX40N,NTT,DURABLES
50,KANSAINER,299.63,340.0,-68.02,H-SC,22.58,223290.0,-46377.0,82707.0,1.64,-17.20,37.04,13.47,142.0,-0.56,1.66,12.35,XY24,NTT,PAINTS


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TCS,3794.03,4998.00,-17.74,X-LC,30.11,268406.0,-27528.0,121427.0,-0.02,-9.30,45.24,31.73,1.0,-0.23,2.00,7.43,X40,BTT,IT
45,INFY,1461.46,2275.00,-10.27,X-LC,56.99,287832.0,26231.0,119393.0,-0.50,10.03,41.48,55.67,2.0,0.22,2.14,16.74,X40,BTT,IT
37,HINDUNILVR,2413.81,2723.00,-17.52,X-LC,24.73,258373.0,-11974.0,46610.0,1.14,-4.43,18.04,12.81,6.0,-0.26,1.92,8.06,XY25,NTT,FMCG
58,NESTLEIND,2268.60,2755.00,-9.30,X-LC,36.56,287668.0,22242.0,34664.0,1.12,8.38,12.05,21.44,9.0,0.64,2.14,15.10,XY25,NTT,FMCG
16,BRITANNIA,4983.33,6446.05,16.31,X-LC,68.82,266547.0,37314.0,29960.0,-0.57,16.28,11.24,29.35,10.0,1.25,1.99,26.65,XY25,ATH,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
91,WHIRLPOOL,1167.49,2270.0,-37.42,M-SC,90.32,206212.0,34591.0,127480.0,-0.34,20.16,61.82,94.43,198.0,0.27,1.54,52.20,XR,NTT,DURABLES
85,UNIONBANK,123.87,163.0,-2.93,M-LC,73.12,169481.0,28641.0,15846.0,0.63,20.34,9.35,31.59,89.0,1.81,1.26,47.22,XY24,NTT,BANKS
3,ADANIPORTS,1103.69,1583.0,1.67,M-LC,52.69,102254.0,23892.0,10144.0,0.76,30.49,9.92,43.43,77.0,2.36,0.76,37.43,XY24,NTT,MISC


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SAIL,130.64,228.00,88.63,M-MC,51.61,226787.0,1825.0,165827.0,-1.44,0.81,73.12,74.53,185.0,0.01,1.69,31.67,XY24,BTT,STEEL
19,CAMS,3643.00,5250.99,6.98,H-SC,81.72,120201.0,18197.0,26829.0,1.90,17.84,22.32,44.14,124.0,0.68,0.90,36.98,X40N,ATH,MISC
48,JIOFIN,310.58,387.00,5.62,M-LC,89.25,226415.0,9009.0,44491.0,3.50,4.14,19.65,24.61,95.0,0.20,1.69,60.98,XY24,BTT,FINANCE
3,ADANIPORTS,1103.69,1583.00,1.67,M-LC,52.69,102254.0,23892.0,10144.0,0.76,30.49,9.92,43.43,77.0,2.36,0.76,37.43,XY24,NTT,MISC
85,UNIONBANK,123.87,163.00,-2.93,M-LC,73.12,169481.0,28641.0,15846.0,0.63,20.34,9.35,31.59,89.0,1.81,1.26,47.22,XY24,NTT,BANKS


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
91,WHIRLPOOL,1167.49,2270.00,-37.42,M-SC,90.32,206212.0,34591.0,127480.0,-0.34,20.16,61.82,94.43,198.0,0.27,1.54,52.20,XR,NTT,DURABLES
92,WIPRO,243.46,420.00,-4.22,M-LC,78.49,164331.0,13386.0,96068.0,-1.29,8.87,58.46,72.51,69.0,0.14,1.22,13.20,XR,NTT,IT
42,INDIAMART,2327.09,4850.92,-46.96,H-SC,93.55,138240.0,14904.0,118859.0,-1.10,12.08,85.98,108.45,122.0,0.13,1.03,37.14,AR,ATH,MISC
57,NATIONALUM,189.63,247.44,-49.40,H-MC,65.59,101140.0,636.0,30008.0,-1.43,0.63,29.67,30.49,97.0,0.02,0.75,33.24,MH,ATH,METALS
35,HEROMOTOCO,4311.81,6039.03,-6.19,H-MC,33.33,151210.0,297.0,60151.0,0.99,0.20,39.78,40.06,102.0,0.00,1.13,23.68,AR,ATH,AUTO


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-51.71,L-SC,91.40,70618.0,-42931.0,82983.0,-2.36,-37.81,117.51,35.27,261.0,-0.52,0.53,72.84,XR,NTT,HOTELS
55,MASFIN,326.60,402.39,-17.80,H-SC,77.42,93405.0,-4575.0,27312.0,0.78,-4.67,29.24,23.21,137.0,-0.17,0.70,35.47,XR,ATH,FINANCE
4,ALKYLAMINE,2347.98,4546.37,4.57,X-SC,98.92,97262.0,-3701.0,98235.0,-0.37,-3.67,101.00,93.63,64.0,-0.04,0.72,44.87,SR,ATH,CHEMICALS
3,ADANIPORTS,1103.69,1583.00,1.67,M-LC,52.69,102254.0,23892.0,10144.0,0.76,30.49,9.92,43.43,77.0,2.36,0.76,37.43,XY24,NTT,MISC
5,APOLLOHOSP,6904.43,8285.00,-19.51,H-LC,53.76,109628.0,6062.0,14646.0,2.92,5.85,13.36,20.00,30.0,0.41,0.82,20.75,X40N,BTT,HEALTHCARE


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,75.41,137.00,7177.78,M-SC,100.00,79827.0,-13983.0,90604.0,-0.99,-14.91,113.50,81.67,242.0,-0.15,0.59,54.59,XR,NTT,CERAMICS
4,ALKYLAMINE,2347.98,4546.37,4.57,X-SC,98.92,97262.0,-3701.0,98235.0,-0.37,-3.67,101.00,93.63,64.0,-0.04,0.72,44.87,SR,ATH,CHEMICALS
49,JPPOWER,18.73,26.20,-31.87,L-SC,97.85,137071.0,-4771.0,61339.0,0.72,-3.36,44.75,39.88,259.0,-0.08,1.02,38.91,XY24,NTT,POWER
89,VIPIND,488.80,489.00,-212.78,H-SC,96.77,81354.0,-13962.0,14001.0,0.12,-14.65,17.21,0.04,154.0,-1.00,0.61,60.46,OX40N,NTT,MISC
66,SAMMAANCAP,170.35,326.00,-172.73,M-SC,95.70,84414.0,-17796.0,111190.0,-1.87,-17.41,131.72,91.37,206.0,-0.16,0.63,37.03,XY25,NTT,FINANCE


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.24
1,25,42.48
2,50,71.06


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.84
LC    34.49
MC    21.62
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.25
X40      12.37
XY25     12.20
X40N     11.88
XR       10.31
OX40N     8.26
AR        7.87
MH        1.68
X5K       1.50
X200      1.42
SR        1.21
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.77
M-SC    16.20
H-LC    12.67
X-LC    12.35
H-MC     8.64
M-LC     8.38
X-MC     7.64
M-MC     4.98
X-SC     4.39
L-SC     2.48
L-LC     1.09
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.33,-1.60,35.56
IT,11.90,-11.02,66.13
FINANCE,7.97,-12.78,53.61
BANKS,7.95,-6.67,50.20
MISC,6.67,-21.06,68.28
PAINTS,5.68,-14.24,36.75
ELECTRICAL,5.06,-3.34,48.10
HEALTHCARE,4.52,-6.10,37.10
AUTO,4.41,-14.20,64.41


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2732252.0,25
XR,1075708.0,13
AR,1012611.0,9
X40,656698.0,9
X40N,569266.0,11
XY25,481583.0,8
OX40N,322358.0,11
SR,180730.0,2
MH,89745.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1996572.0,20
M-SC,1766818.0,21
H-LC,566513.0,11
X-LC,515317.0,7
M-MC,498877.0,4
H-MC,475189.0,7
X-MC,426085.0,6
X-SC,329179.0,5
M-LC,319048.0,6


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,870135.0,6
M-SC,XY24,816825.0,7
H-SC,AR,518963.0,3
M-MC,XY24,386255.0,3
M-SC,XR,330951.0,4
X-LC,X40,240820.0,2
H-SC,X40N,237030.0,4
H-MC,XY24,225180.0,2
H-SC,XR,214708.0,3


In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 38.0 seconds
